In [1]:
# cree une spark session
from pyspark.sql import SparkSession


spark = (
    SparkSession 
    .builder 
    .appName("Streaming from spring-boot") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

In [2]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "telecom_events")
    .option("failOnDataLoss", "false")
    .option("startingOffsets", "earliest")
    .load()
)


In [3]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
wget https://jdbc.postgresql.org/download/postgresql-42.2.27.jar -P jars/

SyntaxError: invalid syntax (923942775.py, line 1)

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, lower, trim, coalesce, lit, date_format
from pyspark.sql.types import *
spark = SparkSession.builder \
    .appName("TelecomPipeline") \
    .config("spark.jars", "jars/postgresql-42.2.27.jar") \
    .getOrCreate()

# Définir le schéma attendu
schema = StructType([
    StructField("event_id", StringType()),
    StructField("timestamp", StringType()),
    StructField("event_type", StringType()),
    StructField("user_id", StringType()),
    StructField("destination_id", StringType()),
    StructField("duration", DoubleType()),
    StructField("volume", LongType()),
    StructField("cell_id", StringType()),
    StructField("technology", StringType()),
    StructField("status", StringType()),
    StructField("error_code", StringType())
])
# Lecture du topic Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "telecom_events") \
    .load()


# Convertir la colonne value
json_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")


In [6]:
from pyspark.sql.functions import lower, trim, coalesce, lit

clean_df = json_df \
    .withColumn("event_type", lower(trim(col("event_type")))) \
    .withColumn("status", coalesce(col("status"), lit("completed"))) \
    .filter(col("event_id").isNotNull() & col("timestamp").isNotNull())
# Sauvegarde en fichier parquet (batch intermédiaire)
clean_df.write.mode("overwrite").parquet("output/telecom_cleaned.parquet")


AnalysisException: 'write' can not be called on streaming Dataset/DataFrame

In [ ]:
# final_df = clean_df.selectExpr("to_json(struct(*)) AS value")

# query = final_df \
#     .writeStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "kafka:9092") \
#     .option("topic", "telecom_cleaned_events") \
#     .option("checkpointLocation", "/tmp/spark_checkpoints") \
#     .start()

# query.awaitTermination()


In [ ]:
# Charger les données nettoyées
rated_df = spark.read.parquet("output/telecom_cleaned.parquet")

# Règles tarifaires simples
tarif_df = rated_df.withColumn(
    "tarif",
    when(col("event_type") == "voice", col("duration") * 0.01)
    .when(col("event_type") == "data", col("volume") / 1024 / 1024 * 0.005)
    .when(col("event_type") == "sms", lit(0.1))
    .otherwise(lit(0.0))
)

rated = tarif_df.withColumn("status_tarif", when(col("tarif") > 0, "rated").otherwise("rejected"))

# Sauvegarde
rated.write.mode("overwrite").parquet("output/rated_events.parquet")


In [ ]:
# Lire les événements tarifiés
rated = spark.read.parquet("output/rated_events.parquet")

# Ajouter colonne mois (cycle de facturation)
rated = rated.withColumn("billing_month", date_format(col("timestamp"), "yyyy-MM"))

# Agrégation des factures
facture_df = rated.groupBy("user_id", "billing_month") \
    .agg({"tarif": "sum"}) \
    .withColumnRenamed("sum(tarif)", "total_ht") \
    .withColumn("tva", col("total_ht") * 0.2) \
    .withColumn("total_ttc", col("total_ht") + col("tva"))


In [ ]:
jdbc_url = "jdbc:postgresql://localhost:5432/telecom_db"
connection_props = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

facture_df.write.jdbc(
    url=jdbc_url,
    table="factures",
    mode="overwrite",
    properties=connection_props
)
